In [167]:
pip install -r ../../requirements.txt

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [168]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [198]:
%run -i ../core_onefile.py

In [199]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
sel_data_path = os.path.join('prover/sel_data.json')
sel_dummy_data_path = os.path.join('shared/sel_dummy_data.json')

=======================  ZK-STATS FLOW =======================

In [200]:
table_path = os.path.join('table.json')
dummy_table_path = os.path.join('shared/dummy_table.json')

def createDummy(table_path, dummy_table_path):
    table_data = json.loads(open(table_path, "r").read())
    # assume all columns have same number of rows
    dummy_table_data ={}
    for col in table_data:
        # not use same value for every column to prevent something weird, like singular matrix
        dummy_table_data[col] = np.round(np.random.uniform(1,10,len(table_data[col])),1).tolist()
    
    json.dump(dummy_table_data, open(dummy_table_path, 'w'))


table_data = json.loads(open(table_path, "r").read())
print(table_data)
x1_vals = table_data['x1']
x2_vals = table_data['x2']
y_vals = table_data['y']
createDummy(table_path, dummy_table_path)
dummy_table_data = json.loads(open(dummy_table_path, "r").read())
dummy_x1_vals = dummy_table_data['x1']
dummy_x2_vals = dummy_table_data['x2']
dummy_y_vals = dummy_table_data['y']


def stacked_x(*args):
    result = np.column_stack((*args, np.ones_like(args[0])))
    return result

x_one = stacked_x(x1_vals, x2_vals)
dummy_x_one = stacked_x(dummy_x1_vals, dummy_x2_vals)

w_vals = np.matmul(np.matmul(np.linalg.inv(np.matmul(x_one.transpose(), x_one)), x_one.transpose()), y_vals)
dummy_w_vals = np.matmul(np.matmul(np.linalg.inv(np.matmul(dummy_x_one.transpose(), dummy_x_one)), dummy_x_one.transpose()), dummy_y_vals)

print("reg fit: ", w_vals)
w_tensor = torch.tensor(w_vals).reshape(1,-1,1)
dummy_w_tensor = torch.tensor(dummy_w_vals).reshape(1,-1,1)


{'x1': [7.1, 3.2, 8.6, 3.5, 0.1, 9.7, 2.3, 5.7, 2.8, 10.0, 6.0, 6.0, 9.1, 1.7, 9.2, 0.2, 7.8, 3.7, 7.0, 2.5, 2.8, 5.9, 7.3, 2.9, 2.9, 3.5, 1.0, 9.7, 4.8, 0.9, 7.1, 3.6, 8.2, 3.0, 7.6, 4.2, 5.2, 8.1, 6.3, 9.3, 8.8, 8.2, 6.7, 4.9, 5.4, 9.8, 5.9, 7.1, 3.9, 9.3], 'x2': [1.1, 0.2, 0.2, 0.9, 1.3, 0.3, 2.4, 2.2, 3.0, 0.7, 0.3, 2.2, 4.6, 4.6, 1.5, 2.5, 1.0, 4.8, 5.0, 3.1, 1.1, 0.6, 3.7, 3.9, 1.9, 3.7, 2.3, 2.2, 3.5, 4.4, 0.0, 0.2, 4.4, 3.4, 2.9, 0.4, 4.0, 1.2, 3.0, 0.1, 4.7, 1.5, 3.2, 2.7, 4.1, 1.5, 2.1, 0.4, 4.3, 0.2], 'y': [18.5, 5.5, 18.2, 9.0, 4.0, 19.5, 11.7, 17.9, 15.3, 20.8, 12.5, 21.5, 32.5, 18.6, 23.9, 7.0, 16.9, 22.9, 31.0, 15.0, 8.5, 8.7, 28.9, 19.7, 12.5, 17.4, 7.2, 25.5, 21.4, 15.7, 15.5, 8.2, 28.2, 19.5, 25.5, 12.5, 20.3, 21.7, 22.1, 19.6, 32.2, 22.4, 20.6, 19.7, 20.8, 21.1, 21.8, 17.7, 21.1, 19.4]}
reg fit:  [1.98835287 3.14238058 0.06413647]


In [201]:
# Verifier/ data consumer side:
class verifier_model(nn.Module):
    def __init__(self):
        super(verifier_model, self).__init__()
        self.w = nn.Parameter(data = dummy_w_tensor, requires_grad = False)

    def forward(self, *args):
        # infer Y from the last parameter
        Y = args[-1]
        X_one = torch.cat((*args[:-1], torch.ones_like(args[0][:, :, -1:])), dim=2)
        X_T = torch.transpose(X_one, 1, 2)
        return (
            torch.sum(torch.abs(X_T @ X_one @ self.w - X_T @ Y)) <= 0.01 * torch.sum(torch.abs(X_T @ Y)),
            self.w
        )
    

verifier_define_calculation(dummy_table_path, ['x1','x2','y'],sel_dummy_data_path,verifier_model, verifier_model_path)

In [202]:
# prover calculates settings, send to verifier

theory_output = w_tensor
print("Theory output: ", theory_output)
class prover_model(nn.Module):
    def __init__(self):
        super(prover_model, self).__init__()
        self.w = nn.Parameter(data = w_tensor, requires_grad = False)

    def forward(self, *args):
        # infer Y from the last parameter
        Y = args[-1]
        X_one = torch.cat((*args[:-1], torch.ones_like(args[0][:, :, -1:])), dim=2)
        X_T = torch.transpose(X_one, 1, 2)
        return (
            torch.sum(torch.abs(X_T @ X_one @ self.w - X_T @ Y)) <= 0.01 * torch.sum(torch.abs(X_T @ Y)),
            self.w
        )
# try scale = [3] for more accuracy
prover_gen_settings(table_path,['x1','x2','y'], sel_data_path, prover_model,prover_model_path, [0], "resources", settings_path)

Theory output:  tensor([[[1.9884],
         [3.1424],
         [0.0641]]], dtype=torch.float64)
==== Generate & Calibrate Setting ====
scale:  [0]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":0,"param_scale":0,"scale_rebase_multiplier":10,"lookup_range":[-23296,18300],"logrows":16,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":11808,"total_assignments":794,"total_const_size":13,"model_instance_shapes":[[1],[1,3,1]],"model_output_scales":[0,0],"model_input_scales":[0,0,0],"module_sizes":{"kzg":[],"poseidon":[11808,[3]],"elgamal":[0,[0]]},"required_lookups":["Abs",{"Div":{"denom":100.0}},{"GreaterThan":{"a":0.0}}],"check_mode":"UNSAFE","version":"7.0.0","num_blinding_factors":null}


In [203]:
# Here verifier & prover can concurrently call setup since all params are public to get pk. 
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
verifier_setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
print("Theory output: ", theory_output)
prover_gen_proof(prover_model_path, sel_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

spawning module 0
spawning module 2


==== setting up ezkl ====


spawning module 0
spawning module 2


Time setup: 6.9944376945495605 seconds
Theory output:  tensor([[[1.9884],
         [3.1424],
         [0.0641]]], dtype=torch.float64)
!@# compiled_model exists? False
!@# compiled_model exists? True
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 2.0
witness result 2 : 3.0
witness result 3 : 0.0
==== Generating Proof ====


spawning module 0
spawning module 2


proof:  {'instances': [[[6145674602038562713, 11796601694033167407, 3132644448460071153, 1431119980703310933], [17187590983289934876, 11857991285122296962, 971807162298867662, 379283799527326290], [3957842973089931008, 9845595232537184463, 786695466761881781, 2995319695946854765], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [6425625360762666998, 7924344314350639699, 14762033076929465436, 2023505479389396574], [415066004289224689, 11886516471525959549, 3696305541684646538, 3035258219084094862], [0, 0, 0, 0]]], 'proof': '0c828afbfd46b7d2e0367d970908232ef25e03dfb576f17cdb32bc0dcbd5aa8701e076c9a1ef5bd9a4d382ec75e4b43759ae1cc75664f807435bfab550b559e00e2d16ec3a018507c227268368bf8c38e6030b51f468f809b72d7c684da329d014910d1586dca58bec67f364ff05fe96b367281b4dcece4b108349d94d53614507cc7603347a3321f40159249cf16ea4841d6732413d172d7381e47efc4438b816aebaf64711766e64d0513d146cd1bbedfdefa7319efda53a849e096a42f0b12548528fedbd9c2c371e67a042adea82c96194b7911e510e

In [204]:
verifier_verify(proof_path, settings_path, vk_path)

num_inputs:  3
prf instances:  [[[6145674602038562713, 11796601694033167407, 3132644448460071153, 1431119980703310933], [17187590983289934876, 11857991285122296962, 971807162298867662, 379283799527326290], [3957842973089931008, 9845595232537184463, 786695466761881781, 2995319695946854765], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [6425625360762666998, 7924344314350639699, 14762033076929465436, 2023505479389396574], [415066004289224689, 11886516471525959549, 3696305541684646538, 3035258219084094862], [0, 0, 0, 0]]]
proof boolean:  1.0
proof result 1 : 2.0
proof result 2 : 3.0
proof result 3 : 0.0
verified
